# Python Tutorial: Understanding Object-Relational Mapping with SQLAlchemy

## Session 6: ORM Solutions / Usage

This session will introduce Object-Relational Mapping (ORM) with SQLAlchemy, a powerful tool for managing databases through Python classes. ORM allows developers to work with databases using native Python code instead of SQL, which can simplify the development process and reduce the likelihood of errors.

### Introduction to Object-Relational Mapping (ORM)

Object-Relational Mapping (ORM) is a programming technique used to convert data between incompatible type systems in object-oriented programming languages. In simpler terms, ORM allows you to interact with your database using Python code instead of SQL queries. This abstraction can significantly speed up development and maintainability.

### Why Use SQLAlchemy?

SQLAlchemy is one of the most popular ORM libraries in the Python ecosystem. It provides a full suite of well-known enterprise-level persistence patterns and is designed for efficient and high-performing database access.



### 1. Installing SQLAlchemy

Before you can start using SQLAlchemy, you need to install it along with a library to interact with your specific database. For MariaDB, you can use the same MariaDB connector as before.


In [1]:
!pip install sqlalchemy mariadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.0 MB/s eta 0:00:00a 0:00:01



### 2. Defining Your Database Models

In SQLAlchemy, the database schema is defined using Python classes. Each class corresponds to a database table, and each attribute of the class corresponds to a column in the table.

#### Example: Mapping a `Truck` Class to a Database Table

Here, we'll create a Python class that maps to a `trucks` table in a MariaDB database.


In [10]:
from sqlalchemy import Column, Integer, String, Date
from sqlalchemy.orm import declarative_base


# Define the base class
Base = declarative_base()

# Define the Truck class
class Truck(Base):
    __tablename__ = 'orm-trucks'

    truck_id = Column(Integer, primary_key=True)
    license_plate = Column(String(15), nullable=False)
    model = Column(String(40), nullable=False)
    capacity_ton = Column(Integer)
    maintenance_date = Column(Date)

    def __repr__(self):
        return f"<Truck(license_plate='{self.license_plate}', model='{self.model}')>"


### 3. Connecting to the Database

You need to create an engine to manage connections to the database. SQLAlchemy uses an engine to interact with the database, which provides a source of database connectivity and behavior.


In [11]:
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
load_dotenv()

# Create an engine that stores data in the local directory's
engine = create_engine(f'mariadb+mariadbconnector://{os.environ["M_USER"]}:{os.environ["M_PW"]}@{os.environ["M_SERVER"]}/{os.environ["M_DB"]}')

### 4. Creating Tables

With SQLAlchemy, you can create your tables in the database directly from your Python code by calling the `create_all` method on the `Base` class.

In [12]:
Base.metadata.create_all(engine)


### 5. Creating and Managing Sessions

To interact with the database, SQLAlchemy uses sessions. You'll need to create a session to add or retrieve data from the database.


In [13]:
from sqlalchemy.orm import sessionmaker

# Create a session
Session = sessionmaker(bind=engine)
session = Session()


### 6. Adding and Querying Data

#### Adding Data

In [14]:
new_truck = Truck(license_plate='XYZ123', model='Volvo FH16', capacity_ton=20)
session.add(new_truck)
session.commit()


#### Querying Data


In [15]:
for truck in session.query(Truck).order_by(Truck.truck_id):
    print(truck)

<Truck(license_plate='XYZ123', model='Volvo FH16')>


## Integrating SQLAlchemy ORM with Flask

In this part of the tutorial, we'll learn how to integrate SQLAlchemy ORM into a Flask application to handle CRUD (Create, Read, Update, Delete) operations for trucks in our logistics system. This allows for more dynamic and interactive web applications.

### 7. Setting Up Flask with SQLAlchemy

To get started, ensure you have Flask and SQLAlchemy installed. If not, install them using pip:

```bash
pip install flask-sqlalchemy

#### Integrating SQLAlchemy with Flask

First, we'll set up our Flask application to work with SQLAlchemy.

```python
from flask import Flask, request, redirect, render_template, url_for
from flask_sqlalchemy import SQLAlchemy
import os
from dotenv import load_dotenv

load_dotenv()

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = f'mariadb+mariadbconnector://{os.environ["M_USER"]}:{os.environ["M_PW"]}@{os.environ["M_SERVER"]}/{os.environ["M_DB"]}'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

# Reuse the Truck model class definition
class Truck(db.Model):
    __tablename__ = 'trucks'

    truck_id = db.Column(db.Integer, primary_key=True)
    license_plate = db.Column(db.String(15), nullable=False)
    model = db.Column(db.String(40), nullable=False)
    capacity_ton = db.Column(db.Integer)
    maintenance_date = db.Column(db.Date)

    def __repr__(self):
        return f"<Truck(license_plate='{self.license_plate}', model='{self.model}')>"
```

### 8. Creating the Database Schema

Run this once to create the database schema based on your models:

```python
db.create_all()
```

### 9. Adding Routes for CRUD Operations

#### Adding a New Truck

```python
@app.route('/add', methods=['GET', 'POST'])
def add_truck():
    if request.method == 'POST':
        license_plate = request.form['license_plate']
        model = request.form['model']
        capacity_ton = request.form.get('capacity_ton', type=int)
        new_truck = Truck(license_plate=license_plate, model=model, capacity_ton=capacity_ton)
        db.session.add(new_truck)
        db.session.commit()
        return redirect(url_for('list_trucks'))
    return render_template('add_truck.html')
```

#### Listing All Trucks

```python
@app.route('/')
@app.route('/trucks')
def list_trucks():
    trucks = Truck.query.all()
    return render_template('list_trucks.html', trucks=trucks)
```

#### Editing a Truck

```python
@app.route('/edit/<int:truck_id>', methods=['GET', 'POST'])
def edit_truck(truck_id):
    truck = Truck.query.get_or_404(truck_id)
    if request.method == 'POST':
        truck.license_plate = request.form['license_plate']
        truck.model = request.form['model']
        truck.capacity_ton = request.form.get('capacity_ton', type=int)
        db.session.commit()
        return redirect(url_for('list_trucks'))
    return render_template('edit_truck.html', truck=truck)
```

#### Deleting a Truck

```python
@app.route('/delete/<int:truck_id>', methods=['POST'])
def delete_truck(truck_id):
    truck = Truck.query.get_or_404(truck_id)
    db.session.delete(truck)
    db.session.commit()
    return redirect(url_for('list_trucks'))
```

### 10. Creating Templates

You will need to create HTML templates for adding, editing, and listing trucks. Here's an example of what the `list_trucks.html` might look like:

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>List of Trucks</title>
</head>
<body>
    <h1>List of Trucks</h1>
    <a href="{{ url_for('add_truck') }}">Add New Truck</a>
    <ul>
        {% for truck in trucks %}
        <li>
            {{ truck.license_plate }} - {{ truck.model }} - Capacity: {{ truck.capacity_ton }} tons
            <a href="{{ url_for('edit_truck', truck_id=truck.truck_id) }}">Edit</a>
            <form action="{{ url_for('delete_truck', truck_id=truck.truck_id) }}" method="post">
                <button type="submit">Delete</button>
            </form>
        </li>
        {% endfor %}
    </ul>
</body>
</html>
```